# Release planning problem with soft constraints



## Formal definition

- Let $k \in \mathbb{N}$  be releases to take into account.
- Let $R = [r_1,r_2,\dotsc,r_n]$ be an array of requierements to be developed and assigned to each *release*
- Let $S = [s_1,s_2,\dotsc,s_m]$ be an array of stakeholders.
- Let $X \in \{1,\dotsc,k+1\}^n$ be an array of integers that represents for each requierement $i$ the number of the release in which it is implemented. If $x_i == k+1$ the requierement $i$ is not implemented
- Let $Y \in \{0,1\}^{k+1 \cdot n}$ be a matrix of binary variables, where $y_{l,i} \in Y / y_{l,i} == 1$ if the requierement $i$ is implemented in release $l$ 
$$
Y = \begin{bmatrix}
y_{1,1} & \cdots & y_{1,n}\\
\vdots & \ddots & \vdots \\
y_{k+1,1} & \cdots & y_{k+1,n} \\
\end{bmatrix}
$$
- Let $E = [e_1,e_2,\dotsc,e_n]$ be an array of efforts associated with each requierement
- Let $p_d$ be the desirable cost of each release
- Let $p_m$ be the max affordable cost of each release
- Let $B= [b_1,b_2,\dotsc,b_m]$ be an array of profit associated to a stakeholder.
- Let $P$ be the precedence relation between $(i,j)$ where $i,j$ are requirements; meaning that $i$ requirement must be implemented if $j$ requirement is implemented.
- Let $A \in \mathbb{R}^{m\cdot n}$ be the priority matrix, where $a_{s,i} \in A / a_{s,i}$ is the priority of the stakeholder $s$ for a requierement $i$. This matrix must be normalized, i.e. the sum of the elements for each row must be 1

The objetive function (OF) is:
$$
max \ \alpha
$$


subject to:
0)


$$
\sum_{s = 1}^{m} \sum_{i=1}^{n} b_s \cdot  a_{s,i} \cdot (|k+1|-x_i) \geq f_1 + \alpha(f_0 - f_1)
$$


1) Release constraint: $x_i$ must contain the number of release in which the requierement is implemented

$$
x_i = \sum_{l=1}^{k+1} l \cdot y_{li} \quad \forall i \in \{1,\dotsc, n\}
$$


2) Implementation constraint: every requierement should be implemented 

$$
\sum_{l=1}^{k+1} y_{li} = 1 \quad \forall i \in \{1,\dotsc ,n\}
$$

3) Cost constaint: The sum of the effort for each requierement in the release must be less or equal than the max affordable cost

$$
\sum_{i=1}^{n} e_i \cdot y_{li} \leq p_m - \alpha(p_m - p_d) \quad \forall l \in \{1, \dotsc ,k+1 \}
$$

4) Precedence constraint: requierement $i$ must be implemented before or in the same release than requierement $j$

$$
x_i \leq x_j \quad \forall (i,j) \in P
$$

In [1]:
from __future__ import division
import pyomo.environ as pyo 
import numpy as np

In [2]:
# Helper functions used to normalize rpp.A
def normalize(x):
    """
    Given a numpy vector X, returns another vector the where the sum of elements is 1
    """
    acc = np.sum(x)
    if acc == 0:
        return x
    return x / acc


def A_normalizate(rpp):
    """
    Given an rpp model with A matrix
    Normalize each row, so the sum of elements per row is 1
    """
    A = np.zeros((rpp.number_of_stakeholders.value, rpp.number_of_requirements.value))
       
    # Assing rpp.A values to A
    for (i, j) in rpp.A.index_set():
        A[i - 1, j - 1] = rpp.A[i, j].value

    # Normalize A
    for i in range(0, A.shape[0]):
        A[i, :] = normalize(A[i, :])

    # Assign A values to rpp.A
    for j in range(0, A.shape[1]):
        for i in range(0, A.shape[0]):
            rpp.A[i + 1, j + 1] = A[i, j]

In [58]:
def abstract_model():
    """
    Creates an abstract model of Rpp problem
    """
    rpp = pyo.AbstractModel()
    rpp.name = "Release planning problem"

    # Model's parameters
    rpp.number_of_requirements = pyo.Param(within=pyo.NonNegativeIntegers)
    rpp.number_of_stakeholders = pyo.Param(within=pyo.NonNegativeIntegers)
    rpp.number_of_releases = pyo.Param(within=pyo.NonNegativeIntegers)
    rpp.pm = pyo.Param(within=pyo.NonNegativeIntegers)
    rpp.pd = pyo.Param(within=pyo.NonNegativeIntegers)
    rpp.f0 = pyo.Param(within=pyo.Reals)
    rpp.f1 = pyo.Param(within=pyo.Reals)
    
    
    # Sets that will be used to iterate over 
    rpp.requirements = pyo.RangeSet(1, rpp.number_of_requirements)
    rpp.stakeholders = pyo.RangeSet(1, rpp.number_of_stakeholders)
    rpp.releases = pyo.RangeSet(1, rpp.number_of_releases)
    
    # Parameters defined over previous defined sets
    rpp.cost = pyo.Param(rpp.requirements)
    rpp.profit = pyo.Param(rpp.stakeholders)
    
    # Relations defined over the cartesian product of sets
    # (i,j) requierement i should be implemented if j is implemented
    rpp.precedence = pyo.Set(within=rpp.requirements * rpp.requirements)
    # (s,i) > 0 if stakeholder s has interest over requierement i
    # This relation is here beacuse the dataset have this information
    # We are using this to initialize matrix A
    rpp.interest = pyo.Set(within=rpp.stakeholders * rpp.requirements)

    # We use this function to assign a requierement priority for each stakeholder
    # This is because the dataset we are using does not have this information
    def A_init(rpp, s, i):
        if (s, i) in rpp.interest:
            return 1
        return 0
    # This parameter needs to be mutable so later on we can normalize it
    rpp.A = pyo.Param(rpp.stakeholders, rpp.requirements, initialize=A_init, mutable=True)

    # Variables
    # Store the number in which the requierement is implemented
    rpp.x = pyo.Var(rpp.requirements, domain=pyo.Integers)
    # y[l,i] == 1 if requierement i is implemented in l release
    rpp.y = pyo.Var(rpp.releases, rpp.requirements, domain=pyo.Binary)
    rpp.a = pyo.Var(bounds=(0, 1))
   
    # Objetive function
    def obj_function_rule(rpp):
        return rpp.a
        #return sum(rpp.profit[s] * sum(rpp.A[s, i] * rpp.number_of_releases - rpp.x[i] for i in rpp.requirements) for s in rpp.stakeholders)
    rpp.OBJ = pyo.Objective(rule=obj_function_rule, sense=pyo.maximize)

    # Constraints
    def profit_constraint_rule(rpp):
        return sum(rpp.profit[s] * sum(rpp.A[s, i] * rpp.number_of_releases - rpp.x[i] for i in rpp.requirements) \
                   for s in rpp.stakeholders) >= rpp.f1 + rpp.a * (rpp.f0 - rpp.f1)
    rpp.profit_constraint = pyo.Constraint(rule=profit_constraint_rule)
    
    
    def release_constraint_rule(rpp, i):
        return sum(rpp.y[l, i] * l for l in rpp.releases) == rpp.x[i]
    rpp.release_constraint = pyo.Constraint(rpp.requirements, rule=release_constraint_rule)

    def implementation_constraint_rule(rpp, i):
        return sum(rpp.y[l, i] for l in rpp.releases) == 1
    rpp.implementation_constraint = pyo.Constraint(rpp.requirements, rule=implementation_constraint_rule)

    def cost_soft_constraint_rule(rpp, l):
        return sum(rpp.cost[i] * rpp.y[l, i] for i in rpp.requirements) <= rpp.pm - rpp.a * (rpp.pm - rpp.pd)
    rpp.cost_soft_constraint = pyo.Constraint(pyo.RangeSet(1, rpp.number_of_releases - 1), rule=cost_soft_constraint_rule)

    def precende_constraint_rule(rpp, i, j):
        return rpp.x[i] <= rpp.x[j]
    rpp.precedence_constraint = pyo.Constraint(rpp.precedence, rule=precende_constraint_rule)
    
    return rpp

In [55]:
from time import perf_counter


rpp = abstract_model()
solver_name = 'cbc'
data_file = "./datasets/rpp_data_soft.dat"
rpp_concrete = rpp.create_instance(data=data_file)
solver = pyo.SolverFactory(solver_name)
solver.options['threads'] = 4


A_normalizate(rpp_concrete)
t0 = perf_counter()
res = solver.solve(rpp_concrete)
t1 = perf_counter()

print(f"Done in {t0-t1}")

res.write()

Done in -0.03101789898937568
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1.0
  Upper bound: 1.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 21
  Number of binary variables: 15
  Number of integer variables: 20
  Number of nonzeros: 1
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics

In [56]:
pyo.display(rpp_concrete.x)

x : Size=5, Index=requirements
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :  None :   3.0 :  None : False : False : Integers
      2 :  None :   2.0 :  None : False : False : Integers
      3 :  None :   1.0 :  None : False : False : Integers
      4 :  None :   3.0 :  None : False : False : Integers
      5 :  None :   1.0 :  None : False : False : Integers


In [59]:
pyo.display(rpp_concrete)

Model Release planning problem

  Variables:
    x : Size=5, Index=requirements
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :   3.0 :  None : False : False : Integers
          2 :  None :   2.0 :  None : False : False : Integers
          3 :  None :   1.0 :  None : False : False : Integers
          4 :  None :   3.0 :  None : False : False : Integers
          5 :  None :   1.0 :  None : False : False : Integers
    y : Size=15, Index=y_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 3) :     0 :   1.0 :     1 : False : False : Binary
        (1, 4) :     0 :   0.0 :     1 : False : False : Binary
        (1, 5) :     0 :   1.0 :     1 : False : False : Binary
        (2, 1) :     0 :   0.0 :     1 : False : False : Binary
        (2, 2) :     0 :   1.0 :     1 : False : False : Binary
 